In [598]:
import pandas as pd
import numpy as np
import nltk
import string
import re

In [600]:
import os

os.getcwd()
#os.chdir('AIT526')

'/home/e5241c72-bd05-44b8-af62-ef15d9868a24'

In [602]:
os.getcwd()

'/home/e5241c72-bd05-44b8-af62-ef15d9868a24'

In [604]:
#import data
df = pd.read_csv('ted_talks_en.csv')
##df.head()

/tmp/ipykernel_125/1333793412.py:2: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,2

Looks like there are way too many columns being brought in. Let's remove all the columns that start with "Unnamed"

In [606]:
df2 = df.loc[:, ~df.columns.str.startswith('Unnamed')]

Great! Now let's narrow down the columns to the string/text based columns

In [609]:
#df2.columns

In [612]:
'''df = df2[['talk_id', 
          'title', 
          'speaker_1', 
          'occupations',
          'about_speakers',
          'topics', 
          'related_talks',
          'description',
          'transcript']]'''
# I think we should remove related talks - Delena
df = df2[['talk_id', 
          'title', 
          'speaker_1', 
          'occupations',
          'about_speakers',
          'topics',
          'description',
          'transcript']]
df.head()


,talk_id,title,speaker_1,occupations,about_speakers,topics,description,transcript
0,1,Averting the climate crisis,Al Gore,{0: ['climate advocate']},{0: 'Nobel Laureate Al Gore focused the world’...,"['alternative energy', 'cars', 'climate change...",With the same humor and humanity he exuded in ...,"Thank you so much, Chris. And it's truly a gre..."
1,92,The best stats you've ever seen,Hans Rosling,{0: ['global health expert; data visionary']},"{0: 'In Hans Rosling’s hands, data sings. Glob...","['Africa', 'Asia', 'Google', 'demo', 'economic...",You've never seen data presented like this. Wi...,"About 10 years ago, I took on the task to teac..."
2,7,Simplicity sells,David Pogue,{0: ['technology columnist']},{0: 'David Pogue is the personal technology co...,"['computers', 'entertainment', 'interface desi...",New York Times columnist David Pogue takes aim...,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,53,Greening the ghetto,Majora Carter,{0: ['activist for environmental justice']},{0: 'Majora Carter redefined the field of envi...,"['MacArthur grant', 'activism', 'business', 'c...","In an emotionally charged talk, MacArthur-winn...",If you're here today — and I'm very happy that...
4,66,Do schools kill creativity?,Sir Ken Robinson,"{0: ['author', 'educator']}","{0: ""Creativity expert Sir Ken Robinson challe...","['children', 'creativity', 'culture', 'dance',...",Sir Ken Robinson makes an entertaining and pro...,Good morning. How are you? (Audience) Good. It...


In [614]:
#df.set_index('talk_id') #not sure if this is needed

Next thing need to do is remove the brackets and leading numbers that are present in some of these columns.
For example, occupations has {0: ['climate advocate']} instead of just 'climate advocate'

In [617]:
#looking at the data types
df.dtypes

talk_id           object
title             object
speaker_1         object
occupations       object
about_speakers    object
topics            object
description       object
transcript        object
dtype: object

In [619]:
#I want to make everything a string, even the ID
df = df.astype('string')

In [621]:
import ast

def str2dict(x):
    if type(x) is str and x[0] == "{":
        return(ast.literal_eval(x))
    else:
        return(x)

def speaker1(x):
    if type(x) is dict:
        return(x[0])
    else:
        return(x)

def list2str(x):
    if type(x) is list:
        value = ""
        for items in x:
            if value == "":
                value = str(items)
            else:
                value = value + ", " + str(items)
        return(value)
    else:
        return(x)

In [623]:
# Clean up "occupations" column
df['occupations'] = df['occupations'].apply(lambda x: str2dict(x))
df['occupations'] = df['occupations'].apply(lambda x: speaker1(x))
df['occupations'] = df['occupations'].apply(lambda x: list2str(x))
df['occupations'] = df['occupations'].apply(lambda x: str(x))
df['occupations'] = df['occupations'].apply(lambda x: x.replace(";",","))

df.head()

,talk_id,title,speaker_1,occupations,about_speakers,topics,description,transcript
0,1,Averting the climate crisis,Al Gore,climate advocate,{0: 'Nobel Laureate Al Gore focused the world’...,"['alternative energy', 'cars', 'climate change...",With the same humor and humanity he exuded in ...,"Thank you so much, Chris. And it's truly a gre..."
1,92,The best stats you've ever seen,Hans Rosling,"global health expert, data visionary","{0: 'In Hans Rosling’s hands, data sings. Glob...","['Africa', 'Asia', 'Google', 'demo', 'economic...",You've never seen data presented like this. Wi...,"About 10 years ago, I took on the task to teac..."
2,7,Simplicity sells,David Pogue,technology columnist,{0: 'David Pogue is the personal technology co...,"['computers', 'entertainment', 'interface desi...",New York Times columnist David Pogue takes aim...,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,53,Greening the ghetto,Majora Carter,activist for environmental justice,{0: 'Majora Carter redefined the field of envi...,"['MacArthur grant', 'activism', 'business', 'c...","In an emotionally charged talk, MacArthur-winn...",If you're here today — and I'm very happy that...
4,66,Do schools kill creativity?,Sir Ken Robinson,"author, educator","{0: ""Creativity expert Sir Ken Robinson challe...","['children', 'creativity', 'culture', 'dance',...",Sir Ken Robinson makes an entertaining and pro...,Good morning. How are you? (Audience) Good. It...


In [625]:
# Clean up "about_speakers" column
df['about_speakers'] = df['about_speakers'].apply(lambda x: str2dict(x))
df['about_speakers'] = df['about_speakers'].apply(lambda x: speaker1(x))
df['about_speakers'] = df['about_speakers'].apply(lambda x: str(x))
df = df.rename(columns={"about_speakers": "about_speaker"})

df.head()

,talk_id,title,speaker_1,occupations,about_speaker,topics,description,transcript
0,1,Averting the climate crisis,Al Gore,climate advocate,Nobel Laureate Al Gore focused the world’s att...,"['alternative energy', 'cars', 'climate change...",With the same humor and humanity he exuded in ...,"Thank you so much, Chris. And it's truly a gre..."
1,92,The best stats you've ever seen,Hans Rosling,"global health expert, data visionary","In Hans Rosling’s hands, data sings. Global tr...","['Africa', 'Asia', 'Google', 'demo', 'economic...",You've never seen data presented like this. Wi...,"About 10 years ago, I took on the task to teac..."
2,7,Simplicity sells,David Pogue,technology columnist,David Pogue is the personal technology columni...,"['computers', 'entertainment', 'interface desi...",New York Times columnist David Pogue takes aim...,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,53,Greening the ghetto,Majora Carter,activist for environmental justice,Majora Carter redefined the field of environme...,"['MacArthur grant', 'activism', 'business', 'c...","In an emotionally charged talk, MacArthur-winn...",If you're here today — and I'm very happy that...
4,66,Do schools kill creativity?,Sir Ken Robinson,"author, educator",Creativity expert Sir Ken Robinson challenges ...,"['children', 'creativity', 'culture', 'dance',...",Sir Ken Robinson makes an entertaining and pro...,Good morning. How are you? (Audience) Good. It...


In [627]:
# Clean up "topics" column
df['topics'] = df['topics'].apply(lambda x: str(x))
df['topics'] = df['topics'].apply(lambda x: x.replace("'",""))
df['topics'] = df['topics'].apply(lambda x: x.replace("[",""))
df['topics'] = df['topics'].apply(lambda x: x.replace("]",""))

df.head()

,talk_id,title,speaker_1,occupations,about_speaker,topics,description,transcript
0,1,Averting the climate crisis,Al Gore,climate advocate,Nobel Laureate Al Gore focused the world’s att...,"alternative energy, cars, climate change, cult...",With the same humor and humanity he exuded in ...,"Thank you so much, Chris. And it's truly a gre..."
1,92,The best stats you've ever seen,Hans Rosling,"global health expert, data visionary","In Hans Rosling’s hands, data sings. Global tr...","Africa, Asia, Google, demo, economics, global ...",You've never seen data presented like this. Wi...,"About 10 years ago, I took on the task to teac..."
2,7,Simplicity sells,David Pogue,technology columnist,David Pogue is the personal technology columni...,"computers, entertainment, interface design, me...",New York Times columnist David Pogue takes aim...,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,53,Greening the ghetto,Majora Carter,activist for environmental justice,Majora Carter redefined the field of environme...,"MacArthur grant, activism, business, cities, e...","In an emotionally charged talk, MacArthur-winn...",If you're here today — and I'm very happy that...
4,66,Do schools kill creativity?,Sir Ken Robinson,"author, educator",Creativity expert Sir Ken Robinson challenges ...,"children, creativity, culture, dance, educatio...",Sir Ken Robinson makes an entertaining and pro...,Good morning. How are you? (Audience) Good. It...


In [632]:
# Export new dataset
df.to_csv("ted_talks_clean.csv", index=False)

In [629]:
#remove the leading and trailing brackets by removing all punctuation. We will need to do this anyway as a part of NLP tokenization.
def remove_punctuation(text):
    for i in string.punctuation:
        text = text.replace(i, '')
    return text

In [595]:
#In every column let's remove the punctuations and all numbers from the string. (Should we only remove leading numbers?)
'''for i in df.columns:
    df[i] = df[i].apply(remove_punctuation)
    df[i] = df[i].str.replace('\d+', '')'''

"for i in df.columns:\n    df[i] = df[i].apply(remove_punctuation)\n    df[i] = df[i].str.replace('\\d+', '')"

Now we can start NLP Analysis and a K-means clustering!